# AI300: Capstone Project Data Extraction

In [1]:
import pymysql
import pandas as pd
import sys
sys.path.append('..')
import config as conf

CURSORCLASS = pymysql.cursors.DictCursor

In [2]:
# Define function to establish MySQL database (local) connection
def initiate_connection():
    try:
        connection = pymysql.connect(host=conf.DB_HOST,
                                     user=conf.DB_USER,
                                     passwd=conf.DB_PWD,
                                     db=conf.DB_NAME,
                                     cursorclass=CURSORCLASS)
        print('[+] Connection Successful')
    except Exception as e:
        print(f'[+] Connection Failed: {e}')
        connection = None

    return connection

In [3]:
# Check connection status
connection = initiate_connection()

[+] Connection Successful


In [4]:
# Schema and table names 
schema = 'capstone'
account_tbl = 'account'
accountusage_tbl = 'account_usage'
churn_tbl = 'churn_status'
customer_tbl = 'customer'
city_tbl = 'city'

In [5]:
sql_query = f"""
    SELECT a.account_id 
            , a.customer_id 
            , a.tenure_months
            , a.num_referrals 
            , a.has_internet_service 
            , a.internet_type 
            , a.has_unlimited_data
            , a.has_phone_service
            , a.has_multiple_lines
            , a.has_premium_tech_support
            , a.has_online_security
            , a.has_online_backup
            , a.has_device_protection
            , a.contract_type
            , a.paperless_billing
            , a.payment_method
            , au.avg_long_distance_fee_monthly
            , au.total_long_distance_fee
            , au.avg_gb_download_monthly
            , au.stream_tv
            , au.stream_movie
            , au.stream_music
            , au.total_monthly_fee
            , au.total_charges_quarter
            , au.total_refunds
            , c.gender
            , c.age
            , c.senior_citizen
            , c.married
            , c.num_dependents
            , c.zip_code
            , city.city_name
            , city.population
            , cs.customer_status 
            , cs.churn_label
            , cs.churn_category
            , cs.churn_reason
    FROM account AS a 
    JOIN account_usage au
    ON a.account_id = au.account_id
    JOIN customer AS c
    ON a.customer_id = c.customer_id
    JOIN city 
    ON c.zip_code = city.zip_code
    JOIN churn_status AS cs
    ON a.customer_id = cs.customer_id
"""

def get_records(sql_query):
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql_query)

        # Connection is not autocommit by default, so we must commit to save changes
        connection.commit()
        
        # Fetch all the records from SQL query output
        results = cursor.fetchall()
        
        # Convert results into pandas dataframe
        df = pd.DataFrame(results)
        
        print(f'Successfully retrieved records')
        
        return df
        
    except Exception as e:
        print(f'Error encountered: {e}')

In [6]:
# Retrieve merged data from databasee
df = get_records(sql_query)
print(f"Shape of dataframe: {df.shape}")

Successfully retrieved records
Shape of dataframe: (7043, 37)


In [7]:
# Export data to CSV file format
df.to_csv('data/customer_churn_v0.csv', index=False)

In [8]:
connection.close()